In [ ]:
# Libraries for plotting
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sgn

In [ ]:
# load processed Nav data
import pandas as pd
nav = pd.read_hdf('../data/interim/nav_converted.h5', 'table')
paros = pd.read_hdf('../data/interim/paros_converted.h5', 'table')
sbe3 = pd.read_hdf('../data/interim/sbe3_converted.h5', 'table')

nadv = pd.read_hdf('../data/interim/nortek_raw.h5', 'table')

# get start time
T0 = nav.head(1).epoch[0]

In [ ]:
# calculate nav heading to yaw conversion

pre_yaw = np.deg2rad(nav.heading.values)
yaw = np.copy(pre_yaw)
for i in range(len(pre_yaw)):
    if pre_yaw[i] > np.pi:
        # flip to negative angle
        yaw[i] = pre_yaw[i] - 2*np.pi
        
nav['yaw'] = yaw

In [ ]:
nav.head(1)

## Nortek to Nav

In [ ]:
nav['nadv_v0'] = np.interp(nav.epoch.values, nadv.epoch.values, nadv.v0.values)
nav['nadv_v1'] = np.interp(nav.epoch.values, nadv.epoch.values, nadv.v1.values)
nav['nadv_v2'] = np.interp(nav.epoch.values, nadv.epoch.values, nadv.v2.values)

nadv.head(1).epoch -T0

In [ ]:
# Filter Nortek data
# ... conv with sync

# Fit Paros to Nav

In [ ]:
def first_order_comp(x, m, c):
    return x*m + c

In [ ]:
nav['pressure'] = np.interp(nav.epoch.values, paros.epoch.values, paros.pressure.values)
nav_slice = sel_by_depth(nav, depthmin=-1550, depthmax=-1450).copy()
params = np.polyfit(nav_slice.pressure.values, nav_slice.depth.values, 1)
print('Fit Parameters: %0.10f %0.10f' % (params[0], params[1]))

# add paros predicted depth
nav['paros_depth'] = nav.pressure*params[0] + params[1]

In [ ]:
# filter paros data
# ... conv with sync

### Calculate Velocities

In [ ]:
# get delta time
dt = np.diff(nav.epoch)
nav["dt"] = np.append(dt, 0.0)
if len(nav.dt) != len(nav.epoch):
    raise ValueError("dt length does not match epochs")
# average samping time
print("mean dt=", np.mean((nav[nav.dt < 1.0]).dt))

In [ ]:
# Paros velocity
paros_vertvel = np.divide(np.diff(nav.paros_depth), dt)
nav['paros_vel'] = np.append(paros_vertvel, 0.0)

## Convenience Functions

In [ ]:
# Data selection functions

#def area_select(df):
#    return df.loc[(df.xrot >= xmin) & (df.xrot <= xmax) & (df.yrot >= ymin) & (df.yrot <= ymax)]

def sel_by_depth(nav, depthmin=-5000, depthmax=-1000):
  return nav[(nav.depth < depthmax) & (nav.depth > depthmin)]

def sel_by_epoch(nav, T0, Tmin=-5000, Tmax=-1000):
  return nav[(nav.epoch < (Tmax+T0)) & (nav.epoch > (Tmin+T0))]

## Visualizing Dive 273 Data

In [ ]:
# nav_slice_plot = sel_by_epoch(nav, T0, Tmin=2960, Tmax=5070) # start of dive 271
nav_273 = sel_by_epoch(nav, T0, Tmin=170000, Tmax=220000) # start of dive 273

fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd1, = ax1.plot(nav_273.epoch.values - T0, nav_273.paros_depth.values,'r', label='paros')
hd2, = ax1.plot(nav_273.epoch.values - T0, nav_273.depth.values, label='nav')

ax1.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth on dive 273')
ax1.grid()

# fig.tight_layout()
# plt.show()
plt.legend(handles=[hd1, hd2])

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd1, = ax1.plot(nav_273.x.values, nav_273.y.values,'r', label='nav')
# hd2, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.depth.values, label='nav')

ax1.set(xlabel='X', ylabel='Y',
       title='Position')
ax1.grid()

## Zooming in on one segment

In [ ]:
# nav_slice_plot = sel_by_epoch(nav, T0, Tmin=2960, Tmax=5070) # start of dive 271
nav_slice_plot = sel_by_epoch(nav, T0, Tmin=191200, Tmax=200000) #Tmax=198870) # start of dive 273

fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd2, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.paros_depth.values,'r', label='paros')
hd1, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.depth.values, label='nav')

ax1.set(xlabel='time [s]', ylabel='depth [m]',
       title='Zoomed dive 273, depth at first survey')
ax1.grid()

plt.legend(handles=[hd1, hd2])

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd1, = ax1.plot(nav_slice_plot.x.values, nav_slice_plot.y.values,'r', label='nav')
# hd2, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.depth.values, label='nav')

ax1.set(xlabel='X', ylabel='Y',
       title='Position')
ax1.grid()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd1, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.pitch.values,'r', label='nav')

ax1.set(xlabel='time', ylabel='pitch [rad]',
       title='Discover pitch angle sign (nose up is positive / +Y->TOSTARBOARD) == NED-frame')
ax1.grid()

ax2 = ax1.twinx()
# hd3, = ax2.plot(nav_slice_plot.epoch.values - T0, yaw, 'g', label='yaw')
hd2, = ax2.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.depth.values, 'm',label='nav')
ax2.set_ylabel('depth [m]', color='m')
ax2.tick_params('y', colors='m')

fig.tight_layout()

In [ ]:
# see the heading angle (check convert to yaw)

fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd1, = ax1.plot(nav_slice_plot.epoch.values-T0, nav_slice_plot.heading.values,'r', label='heading')

ax1.set(xlabel='time [s]', ylabel='heading [deg, [0-360) ]',
       title='Heading')
ax1.grid()

ax2 = ax1.twinx()
hd2, = ax2.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.yaw, 'm', label='yaw')
ax2.set_ylabel('yaw angle [rad, [-pi,pi) ]', color='m')
ax2.tick_params('y', colors='m')

fig.tight_layout()

In [ ]:
# nav_slice_plot = sel_by_epoch(nav, T0, Tmin=172500, Tmax=173800) # start of dive 273

fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)

hd1, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.nadv_v0,'r', label='v0')
hd2, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.nadv_v1,'g', label='v1')
hd3, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.nadv_v2,'b', label='v2')

ax1.set(xlabel='time [s]', ylabel='vel',
       title='Nortek')
ax1.grid()
plt.legend(handles=[hd1, hd2, hd3])

In [ ]:
# nav_slice_plot = sel_by_epoch(nav, T0, Tmin=172500, Tmax=173800) # start of dive 273

fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)

hd1, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.paros_vel,'r', label='v0')

ax1.set(xlabel='time [s]', ylabel='vel',
       title='Paros differenced velocity')
ax1.grid()
plt.legend(handles=[hd1, hd2, hd3])

# Geometric Transformations Functions / Converters

In [ ]:
def e2q(E):
    q = np.array([0.0,0.0,0.0,0.0])
    hroll = E[0]/2.0
    hpitch = E[1]/2.0
    hyaw = E[2]/2.0
    sin_r2 = np.sin(hroll)
    sin_p2 = np.sin(hpitch)
    sin_y2 = np.sin(hyaw)
    cos_r2 = np.cos(hroll)
    cos_p2 = np.cos(hpitch)
    cos_y2 = np.cos(hyaw)
    q[0] = cos_r2 * cos_p2 * cos_y2 + sin_r2 * sin_p2 * sin_y2
    q[1] = sin_r2 * cos_p2 * cos_y2 - cos_r2 * sin_p2 * sin_y2
    q[2] = cos_r2 * sin_p2 * cos_y2 + sin_r2 * cos_p2 * sin_y2
    q[3] = cos_r2 * cos_p2 * sin_y2 - sin_r2 * sin_p2 * cos_y2
    q = q/np.linalg.norm(q)
    if q[0]<0:
        q = -q
    return q

#test
E = np.array([0.0,0.0,0.0])
if np.linalg.norm(np.array([1.0,0,0,0])-e2q(E)) > 1e-10:
    raise ValueError("e2q not working")

def q2R(q):
    # q = [scalar vector]
    w = q[0]
    x = q[1]
    y = q[2]
    z = q[3]

    nrm = np.linalg.norm(q)
    if (abs(nrm) < 0.9):
        disp('QuaternionLib::q2C -- not a unit quaternion');
        R = np.eye(3)
        return R
    nrm = 1./nrm
    w = w*nrm
    x = x*nrm
    y = y*nrm
    z = z*nrm
    x2 = x*x
    y2 = y*y
    z2 = z*z
    w2 = w*w
    xy = 2*x*y
    xz = 2*x*z
    yz = 2*y*z
    wx = 2*w*x
    wy = 2*w*y
    wz = 2*w*z
    R = np.zeros((3,3))
    R[0,0] = w2+x2-y2-z2 
    R[0,1] = xy-wz
    R[0,2] = xz+wy
    R[1,0] = xy+wz
    R[1,1] = w2-x2+y2-z2
    R[1,2] = yz-wx
    R[2,0] = xz-wy
    R[2,1] = yz+wx
    R[2,2] = w2-x2-y2+z2
    return R

# return quaternion equivalent to rotation matrix R
# q is wxyz
# from MIT libbot
def q2e(q):
    E = np.zeros(3)
    roll_a = 2 * (q[0]*q[1] + q[2]*q[3]);
    roll_b = 1 - 2*(q[1]*q[1] + q[2]*q[2]);
    E[0] = np.arctan2(roll_a, roll_b);
    pitch_sin = 2*(q[0]*q[2] - q[3]*q[1]);
    E[1] = np.arcsin(pitch_sin);
    yaw_a = 2*(q[0]*q[3] + q[1]*q[2]);
    yaw_b = 1 - 2*(q[2]*q[2] + q[3]*q[3]);
    E[2] = np.arctan2(yaw_a, yaw_b);
    return E

### Quick tests

In [ ]:
E = np.array([0.0,0.0,0.0])
q2R(e2q(E))

In [ ]:
q = np.array([0.0,1.0,0.0,0.0])
q2e(q)

# Convert Sentry Nav Orientation data to Rotation Matrix

In [ ]:
# caluclate rotation matrices (store as quaternion)
qw = np.array([])
qx = np.array([])
qy = np.array([])
qz = np.array([])
Ei = np.array([0.0,0.0,0.0])

for i in range(len(nav.epoch)):
    Ei[0] = nav.roll.values[i]
    Ei[1] = nav.pitch.values[i]
    Ei[2] = nav.yaw.values[i]
    
    nQb = e2q(Ei)
    np.append(qw, nQb[0])
    np.append(qx, nQb[1])
    np.append(qy, nQb[2])
    np.append(qz, nQb[3])

In [ ]:
# rotate all ADV velocities to local frame

In [ ]:
nav.heading.values[0:10]

In [ ]:
nav.yaw.values[0:10]

In [ ]:
# this is a test

M = np.array([1.0,2.0,3.0])
np.append(M, 4.0)

In [ ]:
qx = np.array([])
print(qx)